<a href="https://colab.research.google.com/github/olaviinha/NeuralAudioStemSeparation/blob/master/NeuralAudioStemSeparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Audio Stem Separation (22kHz sample rate)

This is an extension for [Deezer Spleeter](https://github.com/deezer/spleeter) to be run in Google Colab, using Google Drive as data storage.

`input` value may be
1.   Path to an individual audio file located in your Google Drive.
2.   Path to a directory in your Google Drive containing multiple audio files. Each audio file found in the directory will be individually processed.
3.   Single YouTube URL
4.   Multiple YouTube URLs separated by spaces. Each youtube video will be individually processed.

Input will be separated to 4 stems: vocals, drums, bass and other. Stems will be saved as WAV files to your Google Drive (`output_dir`).

In [ ]:
#@title Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisities.

import os
from google.colab import drive, output
drive.mount('/content/drive')
drive_root = '/content/drive/My Drive/'

# inhagcutils
!pip -q install import-ipynb
!curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

!gsutil -q -m cp -R gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json /content/cfg.json
print('Fetched gs://neural-research/olaviinha/spleeter-configs/custom-4stems-22kHz-z.json')

#@title Install
!apt -q install ffmpeg
!pip -q install spleeter

output.clear()
op(c.ok, 'Setup finished.')


In [ ]:
#@title Separate

input = "https://www.youtube.com/watch?v=tH2w6Oxx0kQ" #@param {type:"string"}
output_dir = "ai/output/spleeter" #@param {type:"string"}
#previews = False #@param {type:"boolean"}

#----------------------------------------------------------------------------#

import os
from os.path import isfile, join
from glob import glob

input_type = check_input_type(input)
output_dir = drive_root+fix_path(output_dir)
tmp_dir = "/content/tmp/"

if not os.path.isdir(tmp_dir):
  !mkdir {tmp_dir}
if not os.path.isdir(output_dir):
  !mkdir "{output_dir}"

if input_type == 'unknown':
  input = drive_root+fix_path(input)
  input_type = check_input_type(input)

if input_type == 'youtube':
  !pip -q install youtube-dl
  !youtube-dl --restrict-filenames -x --no-continue --audio-format wav -o "{tmp_dir}%(title)s.%(ext)s" {input}


from spleeter.separator import Separator
from spleeter.audio.adapter import get_default_audio_adapter
separator = Separator('/content/cfg.json')
if input_type == "dir" or input_type == "youtube":
  if input_type == "dir":
    target = input
  if input_type == "youtube":
    target = dir_tmp
  file_list = list_audio(target)
  for audiofile in file_list:
    separator.separate_to_file(audiofile, output_dir)
    #!spleeter separate -i "{audiofile}" -o "{output_dir}" -p /content/cfg.json
    print('Files saved under directory', basename(audiofile)+'/')
  op(c.ok, 'FIN.')
elif input_type == "file":
  separator.separate_to_file(input, output_dir)
  #!spleeter separate -i "{input}" -o "{output_dir}" -p /content/cfg.json
  op(c.ok, 'FIN.', 'Files have been saved under directory '+basename(input)+'/')
else:
  print("We're not sure what you want to separate. Please check the input field.")